In [ ]:
# Databricks notebook source
# =============================================================
# ✅ UAT MODEL INFERENCE SCRIPT (FINAL VERSION – ALIGNED WITH STAGING LOGIC)
# =============================================================
%pip install xgboost

import mlflow
from mlflow.tracking import MlflowClient
import pandas as pd
import numpy as np
import math
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from pyspark.sql import SparkSession
from datetime import datetime
import warnings
import sys
import os

warnings.filterwarnings("ignore")

# =============================================================
# ✅ CONFIGURATION (FIXED — SAME AS YOUR STAGING SCRIPT)
# =============================================================
UC_CATALOG = "workspace"
UC_SCHEMA = "ml"
MODEL_NAME = f"{UC_CATALOG}.{UC_SCHEMA}.house_price_xgboost_uc2"

# Delta input table for UAT
DELTA_INPUT_TABLE = "workspace.default.house_price_delta"

# Thresholds for validation
MAPE_THRESHOLD = 15.0   # target < 15%
R2_THRESHOLD   = 0.75   # target > 0.75

# Output table
OUTPUT_TABLE = "workspace.default.uat_inference_house_price_xgboost"


# =============================================================
# ✅ INITIALIZATION
# =============================================================
spark = SparkSession.builder.appName("UAT_Inference_Fixed").getOrCreate()
mlflow.set_registry_uri("databricks-uc")
client = MlflowClient()

print("="*80)
print("🚀 UAT MODEL INFERENCE STARTED – FIXED VERSION")
print("="*80)


# =============================================================
# ✅ 1️⃣ Load model from STAGING alias (exact match with staging script)
# =============================================================
def load_staging_model(model_name):
    print(f"\n📌 Loading UC model from alias: @Staging")
    try:
        model_uri = f"models:/{model_name}@Staging"
        model = mlflow.pyfunc.load_model(model_uri)

        mv = client.get_model_version_by_alias(model_name, "Staging")

        print(f"✅ Loaded model version: v{mv.version}")
        print(f"✅ Run ID: {mv.run_id}")
        return model, mv.version, mv.run_id

    except Exception as e:
        raise ValueError(f"❌ Failed to load model from staging: {e}")


# =============================================================
# ✅ 2️⃣ Load Delta table for inference
# =============================================================
def load_data():
    print("\n📌 Loading UAT Delta input data...")
    try:
        df_spark = spark.table(DELTA_INPUT_TABLE)
        df = df_spark.toPandas()

        if "price" not in df.columns:
            raise ValueError("❌ Input table MUST contain 'price' column.")

        X = df.drop(columns=["price"])
        y_true = df["price"]

        print(f"✅ Loaded {len(df)} rows for inference.")
        return df, X, y_true

    except Exception as e:
        raise ValueError(f"❌ Failed to load input table: {e}")


# =============================================================
# ✅ 3️⃣ Run inference
# =============================================================
def run_inference(model, X):
    print("\n📌 Running inference...")
    y_pred = model.predict(X)
    print("✅ Inference complete.")
    return y_pred


# =============================================================
# ✅ 4️⃣ Calculate metrics
# =============================================================
def evaluate(y_true, y_pred):
    mae = mean_absolute_error(y_true, y_pred)
    rmse = math.sqrt(mean_squared_error(y_true, y_pred))
    r2 = r2_score(y_true, y_pred)
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100

    print("\n📊 Evaluation Metrics:")
    print(f"MAE  : {mae:.3f}")
    print(f"RMSE : {rmse:.3f}")
    print(f"R²   : {r2:.3f}")
    print(f"MAPE : {mape:.2f}%")
    return mae, rmse, r2, mape


# =============================================================
# ✅ 5️⃣ Threshold validation (UAT pass/fail)
# =============================================================
def validate(mape, r2):
    if mape <= MAPE_THRESHOLD and r2 >= R2_THRESHOLD:
        print("\n✅ UAT PASSED ✅")
        return "PASSED"
    else:
        print("\n❌ UAT FAILED ❌")
        return "FAILED"


# =============================================================
# ✅ 6️⃣ Log results to Delta table (dedupe included)
# =============================================================
def log_results(model_version, mae, rmse, r2, mape, status):
    result_df = pd.DataFrame([{
        "timestamp": datetime.now(),
        "model_version": int(model_version),
        "mae": mae,
        "rmse": rmse,
        "r2": r2,
        "mape": mape,
        "uat_status": status
    }])

    # Prevent duplicate logs
    try:
        existing = spark.table(OUTPUT_TABLE).toPandas()
        if not existing.empty:
            last = existing.iloc[-1]
            if (
                math.isclose(last.mae, mae, rel_tol=1e-6) and
                math.isclose(last.rmse, rmse, rel_tol=1e-6) and
                math.isclose(last.r2, r2, rel_tol=1e-6) and
                math.isclose(last.mape, mape, rel_tol=1e-6)
            ):
                print("\nℹ️ Metrics unchanged → Skipping log")
                return
    except:
        pass

    spark_df = spark.createDataFrame(result_df)
    spark_df.write.mode("append").saveAsTable(OUTPUT_TABLE)

    print(f"\n📝 Logged results to: {OUTPUT_TABLE}")


# =============================================================
# ✅ MAIN EXECUTION FLOW
# =============================================================
try:
    model, model_version, run_id = load_staging_model(MODEL_NAME)
    df, X, y_true = load_data()
    y_pred = run_inference(model, X)
    mae, rmse, r2, mape = evaluate(y_true, y_pred)
    status = validate(mape, r2)
    log_results(model_version, mae, rmse, r2, mape, status)

    print("\n🎯 UAT INFERENCE COMPLETED SUCCESSFULLY")

except Exception as e:
    print(f"\n❌ UAT ERROR: {str(e)}")
    sys.exit(1)
